In [1]:
from pathlib import Path
from PIL import Image
import pytesseract
import cv2
import os

In [42]:

def preprocess_image(image_path, output_path):
    image = cv2.imread(image_path)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # med_blur = cv2.GaussianBlur(gray,(5,5),0)

    denoised = cv2.fastNlMeansDenoising(gray,None, 10, 7, 21)
    # denoised = cv2.GaussianBlur(gray, (3, 3), 0)  # smaller the kernel size
    # denoised = cv2.bilateralFilter(gray, 9, 75, 75)  # better edges

#     coordinates = cv2.findNonZero(denoised)
#     if coordinates is not None:
#         angle =cv2.minAreaRect(coordinates)[-1]
#         if angle < -45:
#             angle = -(90+angle)
#         else:
#             angle = -angle

#         # Get the center of the image for rotation
#         (h, w) = denoised.shape[:2]
#         center = (w // 2, h // 2)

#         # Compute the rotation matrix
#         matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
#  # Rotate the image to deskew it
#         denoised = cv2.warpAffine(denoised, matrix, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    # binary = cv2.adaptiveThreshold(denoised, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2) #This method calculates the threshold for a pixel using a weighted sum (Gaussian distribution) of the pixel values in its neighborhood, minus a constant. It’s effective when the image has non-uniform lighting.
    # binary = cv2.adaptiveThreshold(denoised, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
    # _, binary = cv2.threshold(denoised, 150, 255, cv2.THRESH_BINARY) #Global thresholding
    _, binary = cv2.threshold(denoised, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    height, width = binary.shape
    # new_size = (int(width * 1.5), int(height * 1.5))  
    # resized = cv2.resize(binary, new_size, interpolation=cv2.INTER_LINEAR)
    # resized = cv2.resize(binary, (int(width / 2), int(height / 2)), interpolation=cv2.INTER_LINEAR)
    resized = cv2.resize(binary, (int(width * 2), int(height * 2)), interpolation=cv2.INTER_CUBIC)

    pil_image = Image.fromarray(resized)
    
    pil_image.save(output_path) # ,dpi=(300, 300)

    # Save the processed image
    # cv2.imwrite(output_path, resized)

input_folder = 'C:/Users/oyku_/Desktop/recipe_pics/recipes'
output_folder = 'C:/Users/oyku_/Desktop/recipe_pics_processed'

for filename in os.listdir(input_folder):
    if filename.endswith('.jpg') or filename.endswith('.jpeg'):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, f'processed_{filename}')
        preprocess_image(input_path, output_path)


In [2]:

def preprocess_image(image_path, output_path):
    # Read the image
    image = cv2.imread(image_path)

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Denoise the image
    denoised = cv2.fastNlMeansDenoising(gray, None, 10, 7, 21)

    # Apply Otsu's thresholding for binarization
    _, binary = cv2.threshold(denoised, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Resize the binary image to twice its size
    height, width = binary.shape
    resized = cv2.resize(binary, (int(width * 2), int(height * 2)), interpolation=cv2.INTER_CUBIC)

    # Convert to PIL Image and save with optional DPI (if needed)
    pil_image = Image.fromarray(resized)
    pil_image.save(output_path, dpi=(300, 300))  # Adjust DPI as necessary

# Define input and output folders
input_folder = 'C:/Users/oyku_/Desktop/recipe_pics/recipes'
output_folder = 'C:/Users/oyku_/Desktop/recipe_pics_processed'

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Process each image in the input folder
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.jpg', '.jpeg')):  # Case-insensitive check for .jpg and .jpeg
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, f'processed_{filename}')
        preprocess_image(input_path, output_path)

print("Image preprocessing completed!")


Image preprocessing completed!


In [3]:
dic = 'C:/Users/oyku_/Desktop/recipe_pics_processed'


In [5]:

files = Path(dic).glob('*jpg')

custom_config =  r'--oem 1 --psm 6'

for file in files:
    print(file)
    print(pytesseract.image_to_string(Image.open(file)))
    print('\n---------------\n')

C:\Users\oyku_\Desktop\recipe_pics_processed\processed_0fb23a52a547b646092f538295c6de42.jpg
Kini Ormpkin Ores

Pagrediant for Filling Prgredoont for Crus

235g. Pumpkin purée 60g. Butter
100g, Sugar 30g. Sugar
1. Egg yolk 1 Egg yolk

12g. Guttermisk
1 Tbap- Flovr

1 tsp. Cinnamon
A pirch of: en\t, pepper. Proune ginger.
around cloves, ground nuceeg.

Recipe fer Crust

t-Hix all ingredients.

z.Wrap dough tight with film paper and chill in the fridge for 15 min,
3.Ro11 it out between 2 oven papers {thickness around 3mm), place bits
ef dough in greased muffin molds. it is common that pastry geta floury,
so don’t worry! Press paatry ingide the mold until you get a Little cup.

1.Mix all ingredients.
2.Fill partially-baked
little crusts.

3.Bake at 170°C far 25min.

110g. Flour

‘

@waterofwhimsy
Recipe by Wiccan Cuisine @ Regan Raiston, 2020


---------------

C:\Users\oyku_\Desktop\recipe_pics_processed\processed_16e2db669a575aeb0fe2b577630ca196.jpg
r " = ineark COOKES <3

Ques. - 2004 F